## This notebook will be used for the Segmenting and Clustering Neighborhoods in Toronto assignment in the Coursera Capstone project

Version 2

*This version will scrap the Toronto postal data from Wikipedia <strong>with the cordinates</strong>, as seen in the fifth questions of the assignment 

In [26]:
import pandas as pd # data analysis library
import numpy as np # library for handeling data in a vectorized manner
import requests # for handeling API requests
from bs4 import BeautifulSoup # for scraping HTML content
import geocoder # import geocoder

print('Libraries are imported')

Libraries are imported


In [13]:
# setup URL and request type
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL)

# setup scraper
scraper = BeautifulSoup(r.content, 'html.parser')
html_table = scraper.find('div', attrs = {'id':'container'})

print('Page is scrapped')

Page is scrapped


In [14]:
# collecting data from scrapped page
torontoPostalCodes = [];
boroughs= [];
neighborhoods = [];
columnNumber = 1;
passValidation = False

for row in scraper.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passValidation = False
            if columnNumber == 1:
                if passValidation == False and cell.string[1].isdigit():
                    torontoPostalCodes.append(cell.string);   
                    columnNumber = 2
                else:
                    continue
            elif columnNumber == 2 :
                if cell.string == 'Not assigned':
                    passValidation = True
                    del torontoPostalCodes[-1]
                    columnNumber = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNumber = 3
            elif columnNumber == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNumber = 1
                
print('Data is collected')

Data is collected


In [37]:
# setup panda datagrame

# define the dataframe columns
column_names = ['PostalCode', 
                'Borough', 
                'Neighborhood',
                'Latitude',
                'Longitude'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

print('Dataframe is initialized')

Dataframe is initialized


In [38]:
lat_lng_coords = None

# fill dataframe with data from scraper
for data in range(0, len(neighborhoods)-1):
    code = torontoPostalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    # setup geocoder and cordinates
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name, 
                                   'Latitude': lat_lng_coords[0], 
                                  'Longitude': lat_lng_coords[1]
                                 }, ignore_index=True)
    
print('Dataframe is filled with data including cordinates')

Dataframe is filled with data including cordinates


In [39]:
# display first five neighbors
neighbors.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,M6A,North York,Lawrence Heights,43.723270,-79.451286
4,M6A,North York,Lawrence Manor,43.723270,-79.451286


In [40]:
# display shape of neighbors
neighbors.shape

(209, 5)